# Attentional Networks in Computer Vision
Prepared by comp411 Teaching Unit (TA Can Küçüksözen) in the context of Computer Vision with Deep Learning Course. Do not hesitate to ask in case you have any questions, contact me at: ckucuksozen19@ku.edu.tr

Up until this point, we have worked with deep fully-connected networks, convolutional networks and recurrent networks using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, on the other hand, most successful image processing methods use convolutional networks. However recent state-of-the-art results on computer vision realm are acquired using Attentional layers and Transformer architectures.

First you will implement several layer types that are used in fully attentional networks. You will then use these layers to train an Attentional Image Classification network, specifically a smaller version of Vision Transformer (VIT) on the CIFAR-10 dataset. The original paper can be accessed via the following link: https://arxiv.org/pdf/2010.11929.pdf

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [99]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [100]:
PYTORCH_ENABLE_MPS_FALLBACK=1 

In [101]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp411/datasets', train=True, download=False,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp411/datasets', train=True, download=False,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp411/datasets', train=False, download=False, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [102]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


# Part II. Barebones Transformers: Self-Attentional Layer

Here you will complete the implementation of the Pytorch nn.module `SelfAttention`, which will perform the forward pass of a self-attentional layer. Our implementation of the SelfAttentional layer will include three distinct fully connected layers which will be responsible of:

1. A fully connected layer, `W_Q`, which will be used to project our input into `queries`
2. A fully connected layer, `W_K`, which will be used to project our input into `keys`
3. A fully connected layer, `W_V`, which will be used to project our input into `values`

After defining such three fully connected layers, and obtain our `queries, keys, and values` variables at the beginning of our forward pass, the following operations should be carried out in order to complete the attentional layer implementation.

1. Seperate each of `query, key, and value` projections into their respective heads. In other words, split the feature vector dimension of each matrix into necessarry number of chunks.

2. Compute the `Attention Scores` between each pair of sequence elements via conducting a scaled dot product operation between every pair of `queries` and `keys`. Note that `Attention Scores` matrix should have the size of `[# of queries , # of keys]`

3. Calculate the `Attention Weights` of each query by applying the non-linear `Softmax` normalization accross the `keys` dimension of the `Attention Scores` matrix.

4. Obtain the output combination of `values` by matrix multiplying `Attention Weights` with `values`

5. Reassemble heads into one flat vector and return the output.

**HINT**: For a more detailed explanation of the self attentional layer, examine the Appendix A of the original ViT manuscript here:  https://arxiv.org/pdf/2010.11929.pdf 

In [115]:
class SelfAttention(nn.Module):
    
    def __init__(self, input_dims, head_dims=128, num_heads=2,  bias=False):
        super(SelfAttention, self).__init__()
        
        ## initialize module's instance variables
        self.input_dims = input_dims
        self.head_dims = head_dims
        self.num_heads = num_heads
        self.proj_dims = head_dims * num_heads
        
        ## Declare module's parameters
        self.W_Q = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_K = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_V = nn.Linear(input_dims, self.proj_dims,bias=bias)

        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        ## Input of shape, [B, N, D] where:
        ## - B denotes the batch size
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D corresponds to model dimensionality
        b,n,d = x.shape
        
        ## Construct queries,keys,values
        q_ = self.W_Q(x)
        k_ = self.W_K(x)
        v_ = self.W_V(x)
        
        ## Seperate q,k,v into their corresponding heads,
        ## After this operation each q,k,v will have the shape: [B,H,N,D//H] where
        ## - B denotes the batch size
        ## - H denotes number of heads
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D//H corresponds to per head dimensionality
        q, k, v = map(lambda z: torch.reshape(z, (b,n,self.num_heads,self.head_dims)).permute(0,2,1,3), [q_,k_,v_])
       
        #########################################################################################
        # TODO: Complete the forward pass of the SelfAttention layer, follow the comments below #
        #########################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        ## Compute attention logits. Note that this operation is conducted as a
        ## batched matrix multiplication between q and k, the output is scaled by 1/(D//H)^(1/2)
        ## inputs are queries and keys that are both of size [B,H,N,D//H]
        ## Output Attention logits should have the size: [B,H,N,N]
        
        alignment_scores = torch.matmul(q, k.transpose(-1, -2)) / np.sqrt(self.head_dims)
    
        ## Compute attention Weights. Note that this operation is conducted as a
        ## Softmax Normalization across the keys dimension. 
        ## Hint: You can apply the Softmax operation across the final dimension

        attention_scores = F.softmax(alignment_scores, dim=2)
        
        ## Compute output values. Note that this operation is conducted as a 
        ## batched matrix multiplication between the Attention Weights matrix and 
        ## the values tensor. After computing output values, the output should be reshaped
        ## Inputs are Attention Weights with size [B, H, N, N], values with size [B, H, N, D//H]
        ## Output should be of size [B, N, D]
        ## Hint: you should use torch.matmul, torch.permute, torch.reshape in that order
        
        attn_out = torch.matmul(attention_scores, v)
        attn_out = attn_out.permute(0,2,1,3)
        attn_out = torch.reshape(attn_out,(b,n,self.proj_dims))
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             
        ################################################################################
    
        return attn_out

After defining the forward pass of the Self-Attentional Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [116]:
def test_self_attn_layer():
    x = torch.zeros((64, 16, 32), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 32
    layer = SelfAttention(32,64,4)
    out = layer(x)
    print(out.size())  # you should see [64,16,256]
test_self_attn_layer()

torch.Size([64, 16, 256])


# Part III. Barebones Transformers: Transformer Encoder Block

Here you will complete the implementation of the Pytorch nn.module `TransformerBlock`, which will perform the forward pass of a Transfomer Encoder Block. You can refer to Figure 1 of the original manuscript of ViT from this link: https://arxiv.org/pdf/2010.11929.pdf in order to get yourself familiar with the architecture.



In [117]:
## Implementation of a two layer GELU activated Fully Connected Network is provided for you below:

class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, bias=True):
        super().__init__()
        
        self.fc_1 = nn.Linear(input_dims, hidden_dims, bias=bias)
        self.fc_2 = nn.Linear(hidden_dims, output_dims, bias=bias)
        
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)
        
    def forward(self, x):
        o = F.relu(self.fc_1(x))
        o = self.fc_2(o)
        return o

In [127]:
## Build from scratch a TransformerBlock Module. Note that the architecture of this
## module follows a simple computational pipeline:
## input --> layernorm --> SelfAttention --> skip connection 
##       --> layernorm --> MLP ---> skip connection ---> output
## Note that the TransformerBlock module works on a single hidden dimension hidden_dims,
## in order to faciliate skip connections with ease. Be careful about the input arguments
## to the SelfAttention block.


class TransformerBlock(nn.Module):
    def __init__(self, hidden_dims, num_heads=4, bias=False):
        super(TransformerBlock, self).__init__()
        
###############################################################
# TODO: Complete the consturctor of  TransformerBlock module  #
###############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****
                
        self.hidden_dims = hidden_dims
        self.num_heads = num_heads
        self.head_dims = self.hidden_dims // self.num_heads
        
        self.norm1 = nn.LayerNorm(self.hidden_dims) 
        self.attention = SelfAttention(self.hidden_dims,head_dims=self.head_dims,num_heads=self.num_heads)
        self.norm2 = nn.LayerNorm(self.hidden_dims)
        self.mlp = MLP(hidden_dims, hidden_dims, hidden_dims)
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################
        
    def forward(self, x):
        
##############################################################
# TODO: Complete the forward of TransformerBlock module      #
##############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        norm1_out = self.norm1(x)
        attention_out = self.attention(norm1_out) + x
        norm2_out = self.norm2(attention_out)
        mlp_out = self.mlp(norm2_out)
        out = mlp_out + attention_out

        return out
        
 # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################

After defining the forward pass of the Transformer Block Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 128).

In [128]:
def test_transfomerblock_layer():
    x = torch.zeros((64, 16, 128), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 128
    layer = TransformerBlock(128,4) # hidden dims size 128, heads size 4
    out = layer(x)
    print(out.size()) 
test_transfomerblock_layer()

torch.Size([64, 16, 128])


# Part IV The Vision Transformer (ViT)

The final implementation for the Pytorch nn.module `ViT` is given to you below, which will perform the forward pass of the Vision Transformer. Study it and get yourself familiar with the API.


In [129]:
class ViT(nn.Module):
    def __init__(self, hidden_dims, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4, bias=False):
        super(ViT, self).__init__()
                
        ## initialize module's instance variables
        self.hidden_dims = hidden_dims
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_trans_layers = num_trans_layers
        self.num_heads = num_heads
        self.image_k = image_k
        self.patch_k = patch_k
        
        self.image_height = self.image_width = image_k
        self.patch_height = self.patch_width = patch_k
        
        assert self.image_height % self.patch_height == 0 and self.image_width % self.patch_width == 0,\
                'Image size must be divisible by the patch size.'

        self.num_patches = (self.image_height // self.patch_height) * (self.image_width // self.patch_width)
        self.patch_flat_len = self.patch_height * self.patch_width
        
        ## Declare module's parameters
        
        ## ViT's flattened patch embedding projection:
        self.linear_embed = nn.Linear(self.input_dims*self.patch_flat_len, self.hidden_dims)
        
        ## Learnable positional embeddings, an embedding is learned for each patch location and the class token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, self.hidden_dims))
        
        ## Learnable classt token and its index among attention sequence elements.
        self.cls_token = nn.Parameter(torch.randn(1,1,self.hidden_dims))
        self.cls_index = torch.LongTensor([0])
        
        ## Declare cascaded Transformer blocks:
        transformer_encoder_list = []
        for _ in range(self.num_trans_layers):
            transformer_encoder_list.append(TransformerBlock(self.hidden_dims, self.num_heads, bias))
        self.transformer_encoder = nn.Sequential(*transformer_encoder_list)
        
        ## Declare the output mlp:
        self.out_mlp = MLP(self.hidden_dims, self.hidden_dims, self.output_dims)
         
    def unfold(self, x, f = 7, st = 4, p = 0):
        ## Create sliding window pathes using nn.Functional.unfold
        ## Input dimensions: [B,D,H,W] where
        ## --B : input batch size
        ## --D : input channels
        ## --H, W: input height and width
        ## Output dimensions: [B,N,H*W,D]
        ## --N : number of patches, decided according to sliding window kernel size (f),
        ##      sliding window stride and padding.
        b,d,h,w = x.shape
        x_unf = F.unfold(x, (f,f), stride=st, padding=p)    
        x_unf = torch.reshape(x_unf.permute(0,2,1), (b,-1,d,f*f)).transpose(-1,-2)
        n = x_unf.size(1)
        return x_unf,n
    
    def forward(self, x):
        b = x.size(0)
        ## create sliding window patches from the input image
        x_patches,n = self.unfold(x, self.patch_height, self.patch_height, 0)
        ## flatten each patch into a 1d vector: i.e. 3x4x4 image patch turned into 1x1x48
        x_patch_flat = torch.reshape(x_patches, (b,n,-1))
        ## linearly embed each flattened patch
        x_embed = self.linear_embed(x_patch_flat)
        
        ## retrieve class token 
        cls_tokens = self.cls_token.repeat(b,1,1)
        ## concatanate class token to input patches
        xcls_embed = torch.cat([cls_tokens, x_embed], dim=-2)
        
        ## add positional embedding to input patches + class token 
        xcls_pos_embed = xcls_embed + self.pos_embedding
        
        ## pass through the transformer encoder
        trans_out = self.transformer_encoder(xcls_pos_embed)
        
        ## select the class token 
        out_cls_token = torch.index_select(trans_out, -2, self.cls_index.to(trans_out.device))
        
        ## create output
        out = self.out_mlp(out_cls_token)
        
        return out.squeeze(-2)

After defining the forward pass of the ViT above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 10).

In [130]:
def test_vit():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size 3,32,32
    model = ViT(hidden_dims=128, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4)
    out = model(x)
    print(out.size()) 
test_vit()

torch.Size([64, 10])


# Part V. Train the ViT

### Check Accuracy
Given any minibatch of input data and desired targets, we can check the classification accuracy of a neural network. 

The check_batch_accuracy function is provided for you below:

In [131]:
def check_batch_accuracy(out, target,eps=1e-7):
    b, c = out.shape
    with torch.no_grad():
        _, pred = out.max(-1) 
        correct = np.sum(np.equal(pred.cpu().numpy(), target.cpu().numpy()))
    return correct, float(correct) / (b)

### Training Loop
As we have already seen in the Second Assignment, in our PyTorch based training loops, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [132]:
def train(network, optimizer, trainloader):
    """
    Train a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - trainloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall training accuracy for the epoch
    """
    print('\nEpoch: %d' % epoch)
    network.train()  # put model to training mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = Variable(inputs.to(device)), targets.to(device)  # move to device, e.g. GPU
            
        outputs = network(inputs)
        loss =  F.cross_entropy(outputs, targets)
            
        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad() 

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()
            
        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()
            
        loss = loss.detach()
        train_loss += loss.item()
        correct_p, _ = check_batch_accuracy(outputs, targets) 
        correct += correct_p
        total += targets.size(0)

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    return 100.*correct/total

### Evaluation Loop
We have also prepared a Evaluation loop in order to determine our networks capabilities in terms of classification accuracy on a given dataset, either the training, or the validation split

In [133]:
def evaluate(network, evalloader):
    """
    Evaluate a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - evalloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall evaluation accuracy for the epoch
    """
    network.eval() # put model to evaluation mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    eval_loss = 0
    correct = 0
    total = 0
    print('\n---- Evaluation in process ----')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(evalloader):
            inputs, targets = inputs.to(device), targets.to(device) # move to device, e.g. GPU
            outputs = network(inputs)
            loss = F.cross_entropy(outputs, targets)
            
            eval_loss += loss.item()
            correct_p, _ = check_batch_accuracy(outputs, targets)
            correct += correct_p
            total += targets.size(0)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (eval_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total

### Overfit a ViT
Now we are ready to run the training loop. A nice trick is to train your model with just a few training samples in order to see if your implementation is actually bug free. 

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `ViT`. 

You also need to define an optimizer that tracks all the learnable parameters inside `ViT`. We prefer to use `Adam` optimizer for this part.

You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

In [134]:
sample_idx_tr = torch.randperm(len(cifar10_train))[:100]
sample_idx_val = torch.randperm(len(cifar10_train))[-100:]

trainset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_tr)
valset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_val)

print("For overfitting experiments, the subset of the dataset that is used has {} sample images".format(len(trainset_sub)))

batch_size_sub = 25
trainloader_sub = torch.utils.data.DataLoader(trainset_sub, batch_size=batch_size_sub, shuffle=True)
valloader_sub = torch.utils.data.DataLoader(valset_sub, batch_size=batch_size_sub, shuffle=False)

print('==> Data ready, batchsize = {}'.format(batch_size_sub))

For overfitting experiments, the subset of the dataset that is used has 100 sample images
==> Data ready, batchsize = 25


In [135]:
learning_rate = 0.002
regularization_val = 1e-6
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

model = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
network= ViT(hidden_dims,input_dims,output_dims, num_trans_layers,num_heads,image_k,patch_k)

optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=regularization_val)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
eval_accs=[]
for epoch in range(15):
    tr_acc = train(network, optimizer, trainloader_sub)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    eval_acc = evaluate(network, valloader_sub)
    print('Evaluation of Epoch {} is completed, Validation accuracy for this epoch is {}'\
              .format(epoch, eval_acc))  
    tr_accs.append(tr_acc)
    eval_accs.append(eval_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final val set accuracy is {}".format(eval_accs[-1]))


Epoch: 0
Loss: 3.518 | Acc: 16.000% (4/25)
Loss: 6.079 | Acc: 10.000% (5/50)
Loss: 6.755 | Acc: 13.333% (10/75)
Loss: 6.064 | Acc: 13.000% (13/100)
Epoch 0 of training is completed, Training accuracy for this epoch is 13.0

---- Evaluation in process ----
Loss: 2.575 | Acc: 12.000% (3/25)
Loss: 2.899 | Acc: 12.000% (6/50)
Loss: 3.174 | Acc: 9.333% (7/75)
Loss: 3.084 | Acc: 10.000% (10/100)
Evaluation of Epoch 0 is completed, Validation accuracy for this epoch is 10.0

Epoch: 1
Loss: 2.655 | Acc: 12.000% (3/25)
Loss: 3.430 | Acc: 14.000% (7/50)
Loss: 3.006 | Acc: 18.667% (14/75)
Loss: 2.880 | Acc: 20.000% (20/100)
Epoch 1 of training is completed, Training accuracy for this epoch is 20.0

---- Evaluation in process ----
Loss: 3.160 | Acc: 4.000% (1/25)
Loss: 2.802 | Acc: 16.000% (8/50)
Loss: 2.783 | Acc: 14.667% (11/75)
Loss: 2.762 | Acc: 16.000% (16/100)
Evaluation of Epoch 1 is completed, Validation accuracy for this epoch is 16.0

Epoch: 2
Loss: 2.731 | Acc: 12.000% (3/25)
Loss: 2.4

## Train the net
By training the four-layer ViT network for three epochs, with untuned hyperparameters that are initialized as below,  you should achieve greater than 50% accuracy both on the training set and the test set:

In [137]:
learning_rate = 0.002
regularization_val = 1e-6
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims,input_dims, output_dims, num_trans_layers, num_heads, image_k, patch_k)
optimizer = optim.Adam(network.parameters(), lr=learning_rate, weight_decay=regularization_val)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
test_accs=[]
for epoch in range(3):
    tr_acc = train(network, optimizer, loader_train)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    test_acc = evaluate(network, loader_test)
    print('Evaluation of Epoch {} is completed, Test accuracy for this epoch is {}'\
              .format(epoch, test_acc))  
    
    tr_accs.append(tr_acc)
    test_accs.append(test_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final test set accuracy is {}".format(test_accs[-1]))


Epoch: 0
Loss: 3.630 | Acc: 4.688% (3/64)
Loss: 4.320 | Acc: 6.250% (8/128)
Loss: 4.282 | Acc: 6.250% (12/192)
Loss: 4.032 | Acc: 7.422% (19/256)
Loss: 3.826 | Acc: 8.750% (28/320)
Loss: 3.685 | Acc: 10.417% (40/384)
Loss: 3.547 | Acc: 11.384% (51/448)
Loss: 3.439 | Acc: 11.328% (58/512)
Loss: 3.334 | Acc: 12.153% (70/576)
Loss: 3.242 | Acc: 12.188% (78/640)
Loss: 3.182 | Acc: 12.074% (85/704)
Loss: 3.107 | Acc: 12.240% (94/768)
Loss: 3.065 | Acc: 12.260% (102/832)
Loss: 3.023 | Acc: 12.612% (113/896)
Loss: 2.966 | Acc: 13.125% (126/960)
Loss: 2.922 | Acc: 13.379% (137/1024)
Loss: 2.889 | Acc: 13.143% (143/1088)
Loss: 2.851 | Acc: 13.628% (157/1152)
Loss: 2.814 | Acc: 14.062% (171/1216)
Loss: 2.785 | Acc: 14.297% (183/1280)
Loss: 2.771 | Acc: 14.509% (195/1344)
Loss: 2.746 | Acc: 15.128% (213/1408)
Loss: 2.734 | Acc: 15.217% (224/1472)
Loss: 2.710 | Acc: 15.690% (241/1536)
Loss: 2.695 | Acc: 15.625% (250/1600)
Loss: 2.677 | Acc: 15.685% (261/1664)
Loss: 2.662 | Acc: 15.741% (272/1728)

Loss: 2.025 | Acc: 26.570% (3622/13632)
Loss: 2.023 | Acc: 26.643% (3649/13696)
Loss: 2.020 | Acc: 26.722% (3677/13760)
Loss: 2.019 | Acc: 26.751% (3698/13824)
Loss: 2.017 | Acc: 26.815% (3724/13888)
Loss: 2.016 | Acc: 26.856% (3747/13952)
Loss: 2.015 | Acc: 26.898% (3770/14016)
Loss: 2.013 | Acc: 26.953% (3795/14080)
Loss: 2.011 | Acc: 27.029% (3823/14144)
Loss: 2.011 | Acc: 27.062% (3845/14208)
Loss: 2.010 | Acc: 27.067% (3863/14272)
Loss: 2.009 | Acc: 27.121% (3888/14336)
Loss: 2.007 | Acc: 27.188% (3915/14400)
Loss: 2.007 | Acc: 27.212% (3936/14464)
Loss: 2.005 | Acc: 27.251% (3959/14528)
Loss: 2.005 | Acc: 27.275% (3980/14592)
Loss: 2.004 | Acc: 27.306% (4002/14656)
Loss: 2.004 | Acc: 27.344% (4025/14720)
Loss: 2.003 | Acc: 27.415% (4053/14784)
Loss: 2.002 | Acc: 27.425% (4072/14848)
Loss: 2.001 | Acc: 27.468% (4096/14912)
Loss: 2.000 | Acc: 27.511% (4120/14976)
Loss: 1.998 | Acc: 27.553% (4144/15040)
Loss: 1.996 | Acc: 27.609% (4170/15104)
Loss: 1.994 | Acc: 27.657% (4195/15168)


Loss: 1.874 | Acc: 31.482% (8422/26752)
Loss: 1.874 | Acc: 31.481% (8442/26816)
Loss: 1.873 | Acc: 31.499% (8467/26880)
Loss: 1.872 | Acc: 31.499% (8487/26944)
Loss: 1.872 | Acc: 31.550% (8521/27008)
Loss: 1.871 | Acc: 31.575% (8548/27072)
Loss: 1.870 | Acc: 31.604% (8576/27136)
Loss: 1.870 | Acc: 31.607% (8597/27200)
Loss: 1.869 | Acc: 31.591% (8613/27264)
Loss: 1.869 | Acc: 31.598% (8635/27328)
Loss: 1.869 | Acc: 31.615% (8660/27392)
Loss: 1.868 | Acc: 31.629% (8684/27456)
Loss: 1.868 | Acc: 31.617% (8701/27520)
Loss: 1.867 | Acc: 31.623% (8723/27584)
Loss: 1.866 | Acc: 31.651% (8751/27648)
Loss: 1.866 | Acc: 31.669% (8776/27712)
Loss: 1.866 | Acc: 31.678% (8799/27776)
Loss: 1.866 | Acc: 31.659% (8814/27840)
Loss: 1.866 | Acc: 31.676% (8839/27904)
Loss: 1.865 | Acc: 31.690% (8863/27968)
Loss: 1.865 | Acc: 31.710% (8889/28032)
Loss: 1.864 | Acc: 31.730% (8915/28096)
Loss: 1.863 | Acc: 31.747% (8940/28160)
Loss: 1.863 | Acc: 31.767% (8966/28224)
Loss: 1.862 | Acc: 31.752% (8982/28288)


Loss: 1.793 | Acc: 34.322% (13619/39680)
Loss: 1.792 | Acc: 34.327% (13643/39744)
Loss: 1.792 | Acc: 34.330% (13666/39808)
Loss: 1.792 | Acc: 34.355% (13698/39872)
Loss: 1.791 | Acc: 34.372% (13727/39936)
Loss: 1.791 | Acc: 34.400% (13760/40000)
Loss: 1.790 | Acc: 34.417% (13789/40064)
Loss: 1.790 | Acc: 34.442% (13821/40128)
Loss: 1.789 | Acc: 34.462% (13851/40192)
Loss: 1.789 | Acc: 34.477% (13879/40256)
Loss: 1.788 | Acc: 34.492% (13907/40320)
Loss: 1.789 | Acc: 34.494% (13930/40384)
Loss: 1.788 | Acc: 34.496% (13953/40448)
Loss: 1.788 | Acc: 34.516% (13983/40512)
Loss: 1.787 | Acc: 34.525% (14009/40576)
Loss: 1.787 | Acc: 34.535% (14035/40640)
Loss: 1.787 | Acc: 34.537% (14058/40704)
Loss: 1.787 | Acc: 34.542% (14082/40768)
Loss: 1.787 | Acc: 34.566% (14114/40832)
Loss: 1.786 | Acc: 34.585% (14144/40896)
Loss: 1.786 | Acc: 34.597% (14171/40960)
Loss: 1.786 | Acc: 34.604% (14196/41024)
Loss: 1.786 | Acc: 34.614% (14222/41088)
Loss: 1.785 | Acc: 34.630% (14251/41152)
Loss: 1.784 | Ac

Loss: 1.562 | Acc: 43.694% (1566/3584)
Loss: 1.561 | Acc: 43.750% (1596/3648)
Loss: 1.557 | Acc: 43.912% (1630/3712)
Loss: 1.558 | Acc: 43.829% (1655/3776)
Loss: 1.557 | Acc: 43.854% (1684/3840)
Loss: 1.555 | Acc: 43.878% (1713/3904)
Loss: 1.554 | Acc: 44.052% (1748/3968)
Loss: 1.557 | Acc: 43.973% (1773/4032)
Loss: 1.557 | Acc: 43.823% (1795/4096)
Loss: 1.559 | Acc: 43.918% (1827/4160)
Loss: 1.558 | Acc: 43.987% (1858/4224)
Loss: 1.559 | Acc: 43.890% (1882/4288)
Loss: 1.556 | Acc: 43.934% (1912/4352)
Loss: 1.554 | Acc: 43.999% (1943/4416)
Loss: 1.554 | Acc: 43.996% (1971/4480)
Loss: 1.552 | Acc: 44.036% (2001/4544)
Loss: 1.554 | Acc: 44.054% (2030/4608)
Loss: 1.552 | Acc: 44.157% (2063/4672)
Loss: 1.550 | Acc: 44.151% (2091/4736)
Loss: 1.551 | Acc: 44.125% (2118/4800)
Loss: 1.548 | Acc: 44.223% (2151/4864)
Loss: 1.546 | Acc: 44.318% (2184/4928)
Loss: 1.546 | Acc: 44.231% (2208/4992)
Loss: 1.545 | Acc: 44.324% (2241/5056)
Loss: 1.545 | Acc: 44.277% (2267/5120)
Loss: 1.545 | Acc: 44.271

...

Loss: 1.353 | Acc: 51.562% (33/64)
Loss: 1.247 | Acc: 58.594% (75/128)
Loss: 1.293 | Acc: 55.208% (106/192)
Loss: 1.261 | Acc: 54.297% (139/256)
Loss: 1.325 | Acc: 52.188% (167/320)
Loss: 1.287 | Acc: 52.865% (203/384)
Loss: 1.296 | Acc: 52.232% (234/448)
Loss: 1.303 | Acc: 51.562% (264/512)
Loss: 1.310 | Acc: 51.562% (297/576)
Loss: 1.305 | Acc: 52.188% (334/640)
Loss: 1.292 | Acc: 53.125% (374/704)
Loss: 1.278 | Acc: 53.776% (413/768)
Loss: 1.279 | Acc: 54.207% (451/832)
Loss: 1.281 | Acc: 54.241% (486/896)
Loss: 1.287 | Acc: 53.958% (518/960)
Loss: 1.279 | Acc: 54.102% (554/1024)
Loss: 1.284 | Acc: 53.860% (586/1088)
Loss: 1.286 | Acc: 53.385% (615/1152)
Loss: 1.291 | Acc: 53.043% (645/1216)
Loss: 1.283 | Acc: 53.516% (685/1280)
Loss: 1.282 | Acc: 53.869% (724/1344)
Loss: 1.289 | Acc: 53.196% (749/1408)
Loss: 1.291 | Acc: 53.125% (782/1472)
Loss: 1.298 | Acc: 52.995% (814/1536)
Loss: 1.291 | Acc: 53.312% (853/1600)
Loss: 1.293 | Acc: 53.125% (884/1664)
Loss: 1.301 | Acc: 52.836% (91

Loss: 1.348 | Acc: 50.430% (6810/13504)
Loss: 1.348 | Acc: 50.450% (6845/13568)
Loss: 1.347 | Acc: 50.455% (6878/13632)
Loss: 1.348 | Acc: 50.445% (6909/13696)
Loss: 1.346 | Acc: 50.501% (6949/13760)
Loss: 1.346 | Acc: 50.506% (6982/13824)
Loss: 1.346 | Acc: 50.518% (7016/13888)
Loss: 1.345 | Acc: 50.538% (7051/13952)
Loss: 1.344 | Acc: 50.557% (7086/14016)
Loss: 1.345 | Acc: 50.575% (7121/14080)
Loss: 1.344 | Acc: 50.559% (7151/14144)
Loss: 1.344 | Acc: 50.591% (7188/14208)
Loss: 1.343 | Acc: 50.610% (7223/14272)
Loss: 1.343 | Acc: 50.614% (7256/14336)
Loss: 1.343 | Acc: 50.653% (7294/14400)
Loss: 1.343 | Acc: 50.643% (7325/14464)
Loss: 1.344 | Acc: 50.619% (7354/14528)
Loss: 1.344 | Acc: 50.630% (7388/14592)
Loss: 1.345 | Acc: 50.600% (7416/14656)
Loss: 1.345 | Acc: 50.564% (7443/14720)
Loss: 1.346 | Acc: 50.561% (7475/14784)
Loss: 1.346 | Acc: 50.552% (7506/14848)
Loss: 1.346 | Acc: 50.530% (7535/14912)
Loss: 1.346 | Acc: 50.541% (7569/14976)
Loss: 1.346 | Acc: 50.519% (7598/15040)


Loss: 1.354 | Acc: 50.445% (13366/26496)
Loss: 1.354 | Acc: 50.448% (13399/26560)
Loss: 1.354 | Acc: 50.454% (13433/26624)
Loss: 1.353 | Acc: 50.468% (13469/26688)
Loss: 1.354 | Acc: 50.467% (13501/26752)
Loss: 1.354 | Acc: 50.462% (13532/26816)
Loss: 1.354 | Acc: 50.454% (13562/26880)
Loss: 1.354 | Acc: 50.438% (13590/26944)
Loss: 1.354 | Acc: 50.444% (13624/27008)
Loss: 1.354 | Acc: 50.447% (13657/27072)
Loss: 1.354 | Acc: 50.431% (13685/27136)
Loss: 1.354 | Acc: 50.426% (13716/27200)
Loss: 1.354 | Acc: 50.407% (13743/27264)
Loss: 1.354 | Acc: 50.403% (13774/27328)
Loss: 1.354 | Acc: 50.398% (13805/27392)
Loss: 1.355 | Acc: 50.393% (13836/27456)
Loss: 1.355 | Acc: 50.378% (13864/27520)
Loss: 1.356 | Acc: 50.359% (13891/27584)
Loss: 1.356 | Acc: 50.354% (13922/27648)
Loss: 1.356 | Acc: 50.346% (13952/27712)
Loss: 1.356 | Acc: 50.364% (13989/27776)
Loss: 1.356 | Acc: 50.356% (14019/27840)
Loss: 1.356 | Acc: 50.355% (14051/27904)
Loss: 1.357 | Acc: 50.354% (14083/27968)
Loss: 1.357 | Ac

Loss: 1.355 | Acc: 50.611% (19888/39296)
Loss: 1.355 | Acc: 50.607% (19919/39360)
Loss: 1.355 | Acc: 50.614% (19954/39424)
Loss: 1.355 | Acc: 50.610% (19985/39488)
Loss: 1.355 | Acc: 50.602% (20014/39552)
Loss: 1.355 | Acc: 50.596% (20044/39616)
Loss: 1.355 | Acc: 50.607% (20081/39680)
Loss: 1.355 | Acc: 50.624% (20120/39744)
Loss: 1.355 | Acc: 50.626% (20153/39808)
Loss: 1.355 | Acc: 50.622% (20184/39872)
Loss: 1.355 | Acc: 50.608% (20211/39936)
Loss: 1.355 | Acc: 50.600% (20240/40000)
Loss: 1.355 | Acc: 50.607% (20275/40064)
Loss: 1.355 | Acc: 50.611% (20309/40128)
Loss: 1.355 | Acc: 50.617% (20344/40192)
Loss: 1.355 | Acc: 50.614% (20375/40256)
Loss: 1.355 | Acc: 50.627% (20413/40320)
Loss: 1.354 | Acc: 50.646% (20453/40384)
Loss: 1.354 | Acc: 50.653% (20488/40448)
Loss: 1.354 | Acc: 50.657% (20522/40512)
Loss: 1.354 | Acc: 50.656% (20554/40576)
Loss: 1.355 | Acc: 50.640% (20580/40640)
Loss: 1.355 | Acc: 50.636% (20611/40704)
Loss: 1.355 | Acc: 50.638% (20644/40768)
Loss: 1.355 | Ac

Loss: 1.287 | Acc: 53.625% (1716/3200)
Loss: 1.288 | Acc: 53.615% (1750/3264)
Loss: 1.287 | Acc: 53.606% (1784/3328)
Loss: 1.287 | Acc: 53.626% (1819/3392)
Loss: 1.287 | Acc: 53.617% (1853/3456)
Loss: 1.285 | Acc: 53.636% (1888/3520)
Loss: 1.284 | Acc: 53.767% (1927/3584)
Loss: 1.286 | Acc: 53.755% (1961/3648)
Loss: 1.283 | Acc: 53.852% (1999/3712)
Loss: 1.282 | Acc: 53.840% (2033/3776)
Loss: 1.282 | Acc: 53.854% (2068/3840)
Loss: 1.284 | Acc: 53.765% (2099/3904)
Loss: 1.283 | Acc: 53.831% (2136/3968)
Loss: 1.284 | Acc: 53.770% (2168/4032)
Loss: 1.285 | Acc: 53.809% (2204/4096)
Loss: 1.286 | Acc: 53.798% (2238/4160)
Loss: 1.285 | Acc: 53.741% (2270/4224)
Loss: 1.284 | Acc: 53.778% (2306/4288)
Loss: 1.283 | Acc: 53.814% (2342/4352)
Loss: 1.280 | Acc: 53.963% (2383/4416)
Loss: 1.278 | Acc: 54.040% (2421/4480)
Loss: 1.277 | Acc: 54.071% (2457/4544)
Loss: 1.279 | Acc: 53.993% (2488/4608)
Loss: 1.279 | Acc: 53.981% (2522/4672)
Loss: 1.277 | Acc: 53.991% (2557/4736)
Loss: 1.279 | Acc: 54.021